In [1]:
import json
import pyvista as pv
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=2)

filename = "/Users/ken/Downloads/montreal_final.json"

In [2]:
with open(filename) as file:
    cm = json.load(file)
    
len(cm["CityObjects"])

28018

In [3]:
totals = {}
for co_id, co in cm["CityObjects"].items():
    if co["type"] in totals:
        totals[co["type"]] += 1
    else:
        totals[co["type"]] = 1
        
for co_type, co_total in totals.items():
    print(str(co_type) + ": " + str(co_total))

Road: 4387
Building: 19430
LandUse: 4120
PlantCover: 80
WaterBody: 1


In [28]:
roads = {}
buildings = {}

for co_id in list(cm["CityObjects"])[0:100]:
    co = cm["CityObjects"][co_id]
    
    if co["type"] == "Road":
        roads[co_id] = {}
        for geom in co["geometry"]:
            if geom["type"] == "MultiLineString":
                geom_line = geom
            if geom["type"] == "MultiSurface":
                geom_surface = geom
                
        lines = []
        lines_vertices = []
        num_lines = 0
        for segment in geom_line["boundaries"]:
            lines.extend([2, 2*num_lines, 2*num_lines+1])
            lines_vertices.extend([cm["vertices"][segment[0]], cm["vertices"][segment[1]]])
            num_lines += 1
        if len(lines) > 0:
            mesh_lines = pv.PolyData(lines_vertices, lines=lines, n_lines=len(geom_line["boundaries"]))
            roads[co_id]["line"] = mesh_lines
        
        surface = []
        surface_vertices = []
        num_triangles = 0
        for triangle in geom_surface["boundaries"]:
            surface.extend([3, 3*num_triangles, 3*num_triangles+1, 3*num_triangles+2])
            surface_vertices.extend([cm["vertices"][triangle[0][0]], cm["vertices"][triangle[0][1]], cm["vertices"][triangle[0][2]]])
            num_triangles += 1
        if len(surface) > 0:
            mesh_surface = pv.PolyData(surface_vertices, surface, len(geom_surface["boundaries"]))
            roads[co_id]["surface"] = mesh_surface
            
    if co["type"] == "Building":
        buildings[co_id] = {}
        

In [31]:
p = pv.Plotter()

for road in roads.values():
    if "surface" in road and "line" in road:
        p.add_mesh(road["surface"], opacity=0.5, show_edges=True)
        p.add_mesh(road["line"], color="red", line_width=2)

p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)